# Dataload

In [ ]:
!pip install roboflow
!pip install pytorch
!pip install opencv-python

In [ ]:

from roboflow import Roboflow
import shutil


rf = Roboflow(api_key="APIKEY_HERE")
project = rf.workspace("ball-tracking-vcxpr").project("court-detection-bxo2j-zts3d")
version = project.version(7)
dataset = version.download("coco")



# Torch Setup

In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models
import cv2
import os
import numpy as np
import json

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

# Torch Dataset

In [ ]:
class KeyPointsDataset(Dataset):
    def __init__(self, data_dir, data_file):
        self.data_dir = data_dir
        with open(data_file, 'r') as f:
            self.data = json.load(f)

        self.transforms = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])
        ])


    def __len__(self):
        return len(self.data["images"])

    def __getitem__(self, idx):
        img_path = self.data["images"][idx]["file_name"]
        img = cv2.imread(os.path.join(self.data_dir, img_path))
        h, w = img.shape[:2]
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = self.transforms(img)

        kps = np.array(self.data["annotations"][idx]["keypoints"])
        kps = kps.astype(np.float32)
        kps[0::3] *= (224 / w)
        kps[1::3] *= (224 / h)

        kps = torch.tensor(kps, dtype=torch.float32)

        return img, kps







In [ ]:
train_data_dir = "Court-Detection-7/train"
val_data_dir = "Court-Detection-7/valid"
file_path = "_annotations.coco.json"
train_dataset = KeyPointsDataset(train_data_dir, os.path.join(train_data_dir, file_path))
val_dataset = KeyPointsDataset(val_data_dir, os.path.join(val_data_dir, file_path))

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=True)

# Create Model

In [ ]:
model = models.resnet50(pretrained=True)

keypoint_counds = 12
model.fc = torch.nn.Linear(model.fc.in_features, keypoint_counds * 3)


In [ ]:
model = model.to(device)

# Train Model

In [ ]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
epoches = 125
for epoch in range(epoches):
    for i, (images, keypoints) in enumerate(train_loader):
        images = images.to(device)
        keypoints = keypoints.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, keypoints)
        loss.backward()
        optimizer.step()

        if (i + 1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{epoches}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')

In [ ]:
torch.save(model.state_dict(), 'keypoints_model_v4.pth')

In [ ]:
model.eval()  # Set the model to evaluation mode
total_mae = 0.0
total_mse = 0.0
num_samples = 0

with torch.no_grad():
    for inputs, targets in val_loader:
        inputs = inputs.to(device)
        targets = targets.to(device)

        # Make predictions
        predictions = model(inputs)

        # Calculate batch metrics
        batch_mae = torch.mean(torch.abs(predictions - targets))
        batch_mse = torch.mean((predictions - targets) ** 2)

        # Accumulate totals
        total_mae += batch_mae.item() * inputs.size(0)
        total_mse += batch_mse.item() * inputs.size(0)
        num_samples += inputs.size(0)

# Calculate overall metrics
avg_mae = total_mae / num_samples
avg_mse = total_mse / num_samples
avg_rmse = torch.sqrt(torch.tensor(avg_mse)).item()

print(f'Average MAE: {avg_mae:.4f}')
print(f'Average MSE: {avg_mse:.4f}')
print(f'Average RMSE: {avg_rmse:.4f}')